# Model Comparison Example

This notebook demonstrates how to classify detected events using Bayesian model comparison. For the source code, see `model_comparison/comparator.py`.

Since `mono-cbp`'s detection algorithm does not exclude events based on their morphology, we need a way to determine which events are consistent with a transit morphology. Other events that are typically either artefacts of poor detrending or spacecraft/detector systematics may pass the previous automated vetting tests, but tend to have characteristic morphologies.

`mono-cbp` fits the aforementioned models (along with a transit model) to each detected event that passes the initial vetting checks using a MCMC approach. Let's take a look at how this works.

## Setup

First, import the necessary modules and configure logging.

In [1]:
from mono_cbp import ModelComparator
from mono_cbp.utils import setup_logging
import os
import logging
%matplotlib inline

setup_logging(log_file=None)

# Configure PyMC to use the logging system
pymc_logger = logging.getLogger("pymc")
pymc_logger.setLevel(logging.INFO)
# Prevent duplicate logging by ensuring PyMC doesn't create its own handlers
for handler in pymc_logger.handlers[:]:
    pymc_logger.removeHandler(handler)

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## Configuration

Define the `model_comparison` configuration parameters.

In [2]:
config = {
    'model_comparison': {
        'tune': 1000,                                 # Number of tuning steps for MCMC
        'draws': 1000,                                # Number of draws for MCMC
        'chains': 4,                                  # Number of chains for MCMC
        'cores': 4,                                   # Number of CPU cores to use
        'target_accept': 0.99,                        # Target acceptance rate for NUTS sampler
        'sigma_threshold': 3,                         # Sigma threshold for outlier rejection
        'aic_threshold': 2,                           # AIC difference threshold, i.e. delta AIC < 2 is ambiguous 
        'rmse_threshold': 1.2,                        # RMSE threshold for ambiguous classification
        'save_plots': True,                           # Enable saving comparison plots
        'plot_dir': 'results/model_comparison_plots'  # Directory for plots
    }
}

## Initialise `ModelComparator`

The `ModelComparator` class allows use to fit a series of models to an event found with `TransitFinder` and classify it based on the best fitting model.

In [3]:
comparator = ModelComparator(config=config)

2025-12-09 03:15:19 - mono_cbp.model_comparison - INFO - Initialised ModelComparator


There are 4 models that `ModelComparator` fits to the input data:

- **Transit model**: This model uses [`exoplanet`](https://docs.exoplanet.codes/en/latest/), as well as input parameters from `TransitFinder` (depth, width, mid-transit time), to fit a simple transit model to the event.
- **Sinusoidal model**: Fits a sinusoid to the event. This is useful because occasionally the detrending does a poor job at removing stellar variability, much of which is inherently sinusoidal
- **Linear model**: Fits a basic straight-line model to the event. Meant to disambiguate low-SNR events (if they get to this point in the vetting).
- **Step/polynomial model**: Tests whether there is a significant (>3 sigma) flux jump in the event. If so, the event is fit with two 2nd order polynomials either side of the jump. If not, the model fits a single polynomial to the event.

Each model is built and sampled using [`PyMC`](https://www.pymc.io/welcome.html).

In [4]:
# Example: Compare a single event from file (TOI-1338b transit)
event_file = 'results/event_snippets/TIC_260128333_6_2.npz'
result = comparator.compare_event(event_file, save_plot=True, plot_dir='results/model_comparison_plots')

2025-12-09 03:15:22 - mono_cbp.model_comparison - INFO - Processing TIC_260128333_6_2.npz
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
2025-12-09 03:17:22 - mono_cbp.model_comparison - INFO - Classification: T, RMSE_transit: 0.94
2025-12-09 03:17:22 - mono_cbp.model_comparison - INFO - Saved comparison plot to results/model_comparison_plots/TIC_260128333_6_2.png


In [5]:
print("SINGLE EVENT RESULTS:")
print("-" * 80)
print(f"Event: {result['filename']}")
print(f"Classification: {result['best_fit']}")
print()
print("AIC Values (lower is better):")
print(f"  Transit:    {result['aic_transit']:.2f}")
print(f"  Sinusoidal: {result['aic_sinusoidal']:.2f}")
print(f"  Linear:     {result['aic_linear']:.2f}")
print(f"  Step:       {result['aic_step']:.2f}")
print()
print("RMSE Values (lower is better):")
print(f"  Transit:    {result['rmse_transit']:.2f}")
print(f"  Sinusoidal: {result['rmse_sinusoidal']:.2f}")
print(f"  Linear:     {result['rmse_linear']:.2f}")
print(f"  Step:       {result['rmse_step']:.2f}")

SINGLE EVENT RESULTS:
--------------------------------------------------------------------------------
Event: TIC_260128333_6_2.npz
Classification: T

AIC Values (lower is better):
  Transit:    -568.17
  Sinusoidal: -524.39
  Linear:     -344.41
  Step:       -521.45

RMSE Values (lower is better):
  Transit:    0.94
  Sinusoidal: 1.40
  Linear:     2.42
  Step:       1.40


### Interpretation

- **T**: Transit model is best fit with ΔAIC_transit ≥ 2, RMSE_transit ≤ 1.2
- **AT**: Transit model is best fit with ΔAIC_transit ≥ 2, RMSE_transit > 1.2 (flagged as ambiguous)
- **Sin**: Sinusiodal model is best fit with ΔAIC_sin ≥ 2, RMSE_sin ≤ 1.2
- **ASin**: Sinusiodal model is best fit with ΔAIC_sin ≥ 2, RMSE_sin > 1.2 (flagged as ambiguous)
- **L**: Linear model is best fit with ΔAIC_line ≥ 2, RMSE_line ≤ 1.2
- **AL**: Linear model is best fit with ΔAIC_line ≥ 2, RMSE_line > 1.2 (flagged as ambiguous)
- **Step**: Step model is best fit with ΔAIC_step ≥ 2, RMSE_step ≤ 1.2
- **AStep**: Step model is best fit with ΔAIC_step ≥ 2, RMSE_step > 1.2 (flagged as ambiguous)
- **A**: Ambiguous (ΔAIC condition is not satisfied for any of the models)

In [6]:
classification = result['best_fit']
if classification == 'T':
    print("Event is classified as transit - high confidence candidate")
elif classification == 'AT':
    print("Event is classified as ambiguous transit - likely candidate")
elif classification == 'Sin':
    print("Event is classified as sinusoid - likely detrending artefact")
elif classification == 'ASin':
    print("Event is classified as ambiguous sinusoid - likely detrending artefact")
elif classification == 'L':
    print("Event is classified as linear - likely spurious event")
elif classification == 'Step':
    print("Event is classified as step/polynomial - likely spurious event")
elif classification == 'AStep':
    print("Event is classified as ambiguous step/polynomial - likely spurious event")
else:
    print("Event is unclassified")

Event is classified as transit - high confidence candidate


We can now see that this event is (correctly) classified as a transit according to our criteria.

Note that the ΔAIC threshold and the RMSE threshold can be altered in `config['model_comparison']`.

## Batch Comparison

To process many events at once, you can use the `compare_events()` method. Let's process the other event in results/event_snippets:

In [7]:
# Process multiple events (either from directory or in-memory list)
event_dir = 'results/event_snippets'
output_dir = 'results'
os.makedirs(output_dir, exist_ok=True)

results_df = comparator.compare_events(
    events_input=event_dir,
    output_file='model_comparison_results.csv',
    output_dir=output_dir
)

print(f"\nProcessed {len(results_df)} events")
results_df.head()

2025-12-09 03:20:19 - mono_cbp.model_comparison - INFO - Processing 3 files from results/event_snippets
2025-12-09 03:20:19 - mono_cbp.model_comparison - INFO - Progress: 0/3
2025-12-09 03:20:19 - mono_cbp.model_comparison - INFO - Processing TIC_260128333_6_1.npz
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
There were 931 divergences after tuning. Increase `target_accept` or reparameterize.
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
2025-12-09 03:22:04 - mono_cbp.model_comparison - INFO - Classification: A, RMSE_transit: 1.08
2025-12-09 03:22:04 - mono_cbp.model_comparison - INFO - Saved comparison plot to results/model_comparison_plots/TIC_260128333_6_1.png
2025-12-09 03:22:04 - mono_cbp.model


Processed 3 events


,filename,best_fit,aic_transit,aic_sinusoidal,aic_linear,aic_step,rmse_transit,rmse_sinusoidal,rmse_linear,rmse_step
0,TIC_260128333_6_1.npz,A,-506.812348,-503.212482,-499.969873,-506.975741,1.077938,1.194325,1.277631,0.863915
1,TIC_260128333_6_2.npz,T,-568.240220,-524.205682,-344.880955,-524.840223,0.940385,1.402658,2.413094,1.367921
2,TIC_260128333_7_1.npz,A,-552.002203,-556.359745,-555.237953,-554.848220,0.812848,0.864265,0.946305,0.923917


## Classification Summary

In [8]:
print("CLASSIFICATION SUMMARY:")
print("-" * 80)
classification_counts = results_df['best_fit'].value_counts()
for classification, count in classification_counts.items():
    pct = count / len(results_df) * 100
    print(f"{classification}: {count} ({pct:.1f}%)")
    
print(f"\nTotal events processed: {len(results_df)}")

CLASSIFICATION SUMMARY:
--------------------------------------------------------------------------------
A: 2 (66.7%)
T: 1 (33.3%)

Total events processed: 3


Model comparison complete! 

**Output files:**
- Vetting results: `results/vetting_results.csv`
- Comparison plots: `results/model_comparison/` (if enabled)


The final step is to visually inspect high-confidence candidates (T and AT classifications).